#### Preamble

Demo of the `ml_adapter` functionality using a simple _numpy_ model

In [1]:
# notebook parameters
# the sdk profile used to connect
PROFILE='_default_'
LOG_LEVEL='INFO'
WS_NAME='myFirstDemo'
WS_VERSION='1.0.1'
PLUG_NAME='myFirstDemo'
PLUG_VERSION='1.0.1'

In [2]:
# setup INFO logging to see http requests made.
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S'
)

# An ML adapter demo

This demo show how _ml adapters_ allow to expose Machine Learning models in webscripts and plugs.

We use a simple function as a model in this demo, as it is not about the training and testing of models itself.



## models as functions

The simplest models are just functions that take numerical data (normally numpy tensors) in. 

In real-world cases, these models would be provided by libraries such as `scikit-learn` or `pytorch`.


In [3]:
def doubler(x):
  return 2 * x

In [4]:
doubler(3)

6

In [5]:
doubler([3,4,5,6])

[3, 4, 5, 6, 3, 4, 5, 6]

In [6]:
import numpy as np
doubler(np.array([3,4,5,6]))

array([ 6,  8, 10, 12])

In [7]:
a_tensor=np.array([[[3],[4],[5],[6]],[[3],[4],[5],[6]]])
display('a_tensor.shape', a_tensor.shape)
result = doubler(a_tensor)
display('result', result)
display('result.shape',result.shape)

'a_tensor.shape'

(2, 4, 1)

'result'

array([[[ 6],
        [ 8],
        [10],
        [12]],

       [[ 6],
        [ 8],
        [10],
        [12]]])

'result.shape'

(2, 4, 1)

In [8]:
class MyModel():
    def __init__(self, multiplier=2.14):
        self.multiplier = np.array(multiplier)

    def __call__(self, x, y=np.array([1,2,3])):
        return { 
            'main': x + self.multiplier * y, 
            'multiplier': self.multiplier 
        }

In [9]:
my_model=MyModel(-2.3)

display(my_model(np.array([2,3,4])))
display(my_model(np.array([2,3,4]),np.array([3,5,3])))
                    

{'main': array([-0.3, -1.6, -2.9]), 'multiplier': array(-2.3)}

{'main': array([-4.9, -8.5, -2.9]), 'multiplier': array(-2.3)}

#### ML adapter as protocol adapter
ML Adapters take care of the protocol conversion from REST to the function interface (e.g. numpy arrays)

We support the following serialization protocols:
* `V1` : https://kserve.github.io/website/master/modelserving/data_plane/v1_protocol
* `V2` : https://github.com/kserve/open-inference-protocol/tree/main 

In [10]:
from ml_adapter.numpy import V1NumpyModelAdapter

In [11]:
adapter = V1NumpyModelAdapter(model=doubler)
await adapter.call({'instances': [2,3,4]})

{'predictions': [4, 6, 8]}

In [12]:
adapter = V1NumpyModelAdapter(model=MyModel(3.14))
await adapter.call({
    'instances': [{'x':[2,3,4], 'y':[3,4,0]}]
}) 

{'outputs': {'main': [[11.42, 15.56, 4.0]], 'multiplier': 3.14}}

# ML adapter as model serializer.

ML adapter also take care of serializing the model.

Currently the following options are available:
* `joblib` creates `model.joblib` or `model.joblib.gz` files
* `dill` creates `model.dill` files
* custome serialization to `model.sav` files: requires registering a `model_class` that has a `save` and `load` method
* no serialization: requires that the model as a argument when creating the adapter (e.g. in the webscript or plug)



In [13]:
# when creating a model, a folder where the assets are stored needs to be present
LOC='demo_webscript'
! rm -fr demo_webscript

In [14]:
adapter = V1NumpyModelAdapter(location=LOC, model=doubler)
adapter.model_class

In [15]:
await adapter.save()

Lets have a look at what is saved  in the `demo_webscript` folder

In [16]:
adapter = await V1NumpyModelAdapter(location=LOC).load()
adapter.model(66)

132

In [17]:
await adapter.call({'instances': [2,3,4]})

{'predictions': [4, 6, 8]}

In [18]:
adapter.as_webscript({ 'name': WS_NAME , 'version': WS_VERSION})
await adapter.save()

In [19]:
# you can save the webscript archive here, and upload it via console. Alternatively, use the sdk as below.
archive_loc = await adapter.save_archive()
archive_loc

PosixPath('demo_webscript.tar.gz')

## Using the the new python sdk

In [21]:
from waylay.sdk import WaylayClient
from waylay.sdk.exceptions import RestResponseError

In [22]:
client = WaylayClient.from_profile()
client

<WaylayClient(services=[gateway,rules,registry],tools=[ml_tool],config={"credentials": {"type": "client_credentials", "api_key": "fc29ca8f37544723fc39d908", "api_secret": "********", "gateway_url": "https://api-aws-dev.waylay.io", "accounts_url": null}, "profile": "_default_", "settings": {}})>

### MLAdapter tool in waylay.sdk
When the `waylay-sdk-ml-adapter` module is loaded, a `waylay.ml_adapter.sdk.MLTool` tool plugin is available.
This tool assists in the creation of _plugs_ and _webscripts_ that wrap ML models using an MLAdapter. 

In [23]:
from ml_adapter.sdk.tool import MLTool
client.ml_tool
print(client.ml_tool.title)
print(client.ml_tool.description)

ML Adapter Tool

Helps creating waylay webscripts and plugs that wrap a machine learning model.



In [24]:
# remove the webscript if it already exists
try:
    await client.registry.webscripts.remove_versions(WS_NAME)
except RestResponseError as exc:
    logging.info(exc.response)

2024-06-11 09:59:58 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/accounts/v1/tokens?grant_type=client_credentials "HTTP/1.1 200 OK"
2024-06-11 09:59:58 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo "HTTP/1.1 202 Accepted"


In [25]:
# update adapter for webscript and deploy
ref = await client.ml_tool.create_webscript(adapter, name=WS_NAME, version=WS_VERSION, draft=True)
ref

2024-06-11 10:00:02 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/webscripts/?draft=true&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': 'Building and deploying webscript myFirstDemo@1.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX'}},
 'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2024-06-11T08:00:02.751Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2024-06-11T08:00:02.764Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T08:00:02.764Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX',
     '740799ef-d515-4704-8718-903851c9899e$z10KIXR2W24RCPc80Wr0t',
     '740799ef-d515-4704-8718-903851c9899e$BgbLl1bxgJ1ErMyAPPoC1']}],
  'status': 'pending',
  'runtime': {'dep

In [26]:
# wait until the webscript is build, deployed and verified to be running ...
ref = await client.ml_tool.wait_until_ready(ref)
ref

2024-06-11 10:00:03 INFO     Waiting for myFirstDemo@1.0.1 to be ready:
2024-06-11 10:00:03 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX&children=true
2024-06-11 10:00:04 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX&children=true "HTTP/1.1 200 OK"
2024-06-11 10:00:04 INFO     ack: Listening to events of jobs dependent on job 740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX
2024-06-11 10:00:04 INFO     myFirstDemo@1.0.1 build: active
2024-06-11 10:00:04 INFO     myFirstDemo@1.0.1 verify: waiting-children
2024-06-11 10:00:04 INFO     myFirstDemo@1.0.1 deploy: waiting-children
2024-06-11 10:00:23 INFO     myFirstDemo@1.0.1 build: completed
{'data': {'returnvalue': {'digest': '468881c8d9579ed0c08b6c8c06d0e52b8a96ee28bfa94086bad90eaf2bf593fb', 'log': [], 'status': 'succ

{'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2024-06-11T08:00:02.751Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2024-06-11T08:00:02.764Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T08:00:02.764Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX',
     '740799ef-d515-4704-8718-903851c9899e$z10KIXR2W24RCPc80Wr0t',
     '740799ef-d515-4704-8718-903851c9899e$BgbLl1bxgJ1ErMyAPPoC1']}],
  'status': 'running',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'web-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': True,
  'webscript': {'name': 'myFirstDemo',
   'version': '1.0.1',
   'runtime': 'web-python3',
   'metadata': {},
   'private': True,
   'allowHmac': True},
  'secret': 'p28LTqX0K56wyIwYwWMYBMeriCMr/QZ76E3naHUdctU='},
 '_links': {'invoke': {'href': 'h

In [27]:
# test the webscript invocation
await client.ml_tool.test_webscript(ref, [23.4])

2024-06-11 10:00:36 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/functions/v1/740799ef-d515-4704-8718-903851c9899e/myFirstDemo "HTTP/1.1 200 OK"


[46.8]

In [28]:
# if succesful, we can _publish_ the webscripts (available on its normal endpoint, any changes will require a new version)
await client.ml_tool.publish(ref)

2024-06-11 10:00:40 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions/1.0.1/publish "HTTP/1.1 201 Created"


{'message': 'Published webscript myFirstDemo@1.0.1',
 'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2024-06-11T08:00:02.751Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2024-06-11T08:00:40.844Z',
  'updates': [{'operation': 'publish',
    'at': '2024-06-11T08:00:40.844Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9'},
   {'operation': 'create',
    'at': '2024-06-11T08:00:02.764Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX',
     '740799ef-d515-4704-8718-903851c9899e$z10KIXR2W24RCPc80Wr0t',
     '740799ef-d515-4704-8718-903851c9899e$BgbLl1bxgJ1ErMyAPPoC1']}],
  'status': 'running',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'web-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': False,
  'webscript': {'name': 'myFirstDemo',
   'version': '1.0.1',
   'runt

In [31]:
# use the regular sdk client e.g. inspect versions
await client.registry.webscripts.list_versions(WS_NAME, select_path='entities')

2024-06-11 10:01:09 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions "HTTP/1.1 200 OK"


[_Model(createdBy='users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9', createdAt='2024-06-11T08:00:02.751Z', updatedBy='users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9', updatedAt='2024-06-11T08:00:40.844Z', updates=[_Model(operation='publish', at='2024-06-11T08:00:40.844Z', by='users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9'), _Model(operation='create', at='2024-06-11T08:00:02.764Z', by='users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9', comment='', jobs=['740799ef-d515-4704-8718-903851c9899e$9DlidM4oOe5vfJO2HukHX', '740799ef-d515-4704-8718-903851c9899e$z10KIXR2W24RCPc80Wr0t', '740799ef-d515-4704-8718-903851c9899e$BgbLl1bxgJ1ErMyAPPoC1'])], status='running', runtime=_Model(deprecated=False, upgradable=False, name='web-python3', version='0.2.0'), deprecated=False, draft=False, webscript=_Model(name='myFirstDemo', version='1.0.1', runtime='web-python3', metadata=_Model(), private=True, allowHmac=True), secret='p28LTqX0K56wyIwYwWMYBMeriCMr/QZ76E3naHUdctU=', _links={'invoke': {'href': 'https://api-aws-dev.wayl

In [32]:
await client.ml_tool.remove(ref)

2024-06-11 10:01:16 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions/1.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': 'Deleting webscript myFirstDemo@1.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$YQE9Y693vabremtRcf0dO&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$YQE9Y693vabremtRcf0dO'}},
 'versions': ['1.0.1']}

In [33]:
await client.registry.webscripts.list_versions(WS_NAME)

2024-06-11 10:01:18 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions "HTTP/1.1 200 OK"


_Model(count=0, entities=[], limit=20)

## Alternative: use `curl` to upload a webscript archive
Alternatively, you can use the archive directly to create the webscript via a `curl` command or the upload feature of the _waylay console_.
Execute the commands shown below in a separate CLI terminal.

In [35]:
print(f'''
# upload webscript
curl -u $WU https://api-aws-dev.waylay.io/registry/v2/webscripts --data-binary @{archive_loc} -H 'content-type: application/tar+gzip'
# check status
curl -u $WU f'https://api-aws-dev.waylay.io/registry/v2/webscripts/{WS_NAME}/versions/{WS_VERSION}'
# check events
curl -u $WU 'https://api-aws-dev.waylay.io/registry/v2/jobs/events' -n
# invoke webscript
INVOKE_URL='{ref["_links"]["invoke"]["href"]}'
curl -u $WU $INVOKE_URL -X POST -d '{{ "instances" : [[[[39849,59995]]]] }}'
# inspect if anything went wrong:
curl -s -u $WU f'https://api-aws-dev.waylay.io/registry/v2/webscripts/{WS_NAME}/versions/0.0.1/jobs' | jq

''')


# upload webscript
curl -u $WU https://api-aws-dev.waylay.io/registry/v2/webscripts --data-binary @demo_webscript.tar.gz -H 'content-type: application/tar+gzip'
# check status
curl -u $WU f'https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions/1.0.1'
# check events
curl -u $WU 'https://api-aws-dev.waylay.io/registry/v2/jobs/events' -n
# invoke webscript
INVOKE_URL='https://api-aws-dev.waylay.io/functions/v1/740799ef-d515-4704-8718-903851c9899e/myFirstDemo'
curl -u $WU $INVOKE_URL -X POST -d '{ "instances" : [[[[39849,59995]]]] }'
# inspect if anything went wrong:
curl -s -u $WU f'https://api-aws-dev.waylay.io/registry/v2/webscripts/myFirstDemo/versions/0.0.1/jobs' | jq




Lets have a look at console at https://console-aws.dev.waylay.io/webscripts?query=myFirst 

# ML Adapter in plugs

Alternatively, you can deploy your model as a _plug_ for the waylay rule engine.

In [37]:
# when creating a model, a folder where the assets are stored needs to be present
PLUG_LOC='demo_plug'
!rm -fr 'demo_plug' 'demo_plug.tar.gz'

In [38]:
adapter = V1NumpyModelAdapter(location=PLUG_LOC, model=doubler)
adapter.as_plug({ 'name': PLUG_NAME })

In [40]:
archive_loc = await adapter.save_archive()
archive_loc

PosixPath('demo_plug.tar.gz')

In [42]:
try:
  await client.registry.plugs.remove_versions(PLUG_NAME, query={'force':True})
except RestResponseError as exc:
    print(exc)

2024-06-11 10:05:45 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/plugs/myFirstDemo?force=true "HTTP/1.1 404 Not Found"


ApiError('Error response.')
Status: 404
Reason: Not Found
Response headers: Headers({'access-control-allow-origin': '*', 'content-type': 'application/json; charset=utf-8', 'content-length': '50', 'date': 'Tue, 11 Jun 2024 08:05:45 GMT', 'x-envoy-upstream-service-time': '8', 'server': 'istio-envoy'})
Response content: <bytes: len=50>
Response data: statusCode=404 error="No plug 'myFirstDemo'"


In [43]:
ref = await client.ml_tool.create_plug(adapter)

2024-06-11 10:05:51 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/plugs/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


In [44]:
ref = await client.ml_tool.wait_until_ready(ref)
ref

2024-06-11 10:05:52 INFO     Waiting for myFirstDemo@0.0.1 to be ready:
2024-06-11 10:05:52 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$HoLoBIHR73NQKzlGJ0w64&children=true
2024-06-11 10:05:52 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$HoLoBIHR73NQKzlGJ0w64&children=true "HTTP/1.1 200 OK"
2024-06-11 10:05:52 INFO     ack: Listening to events of jobs dependent on job 740799ef-d515-4704-8718-903851c9899e$HoLoBIHR73NQKzlGJ0w64
2024-06-11 10:05:52 INFO     myFirstDemo@0.0.1 build: active
2024-06-11 10:05:52 INFO     myFirstDemo@0.0.1 verify: waiting-children
2024-06-11 10:05:52 INFO     myFirstDemo@0.0.1 deploy: waiting-children
2024-06-11 10:06:08 INFO     myFirstDemo@0.0.1 build: completed
{'data': {'returnvalue': {'digest': 'f84975db0865f814fa39a14fb5aef9bc0aee3da8dff7f978e5e8af26a6e38ea6', 'log': [], 'status': 'succ

{'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2024-06-11T08:05:51.104Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2024-06-11T08:05:51.126Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T08:05:51.126Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$HoLoBIHR73NQKzlGJ0w64',
     '740799ef-d515-4704-8718-903851c9899e$KdOaX99-X75fYEUMhrzRs',
     '740799ef-d515-4704-8718-903851c9899e$3tYiRss-LVwzV4wfIbTpA']}],
  'status': 'running',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'plug-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': False,
  'plug': {'name': 'myFirstDemo',
   'version': '0.0.1',
   'runtime': 'plug-python3',
   'metadata': {'tags': [{'name': 'MLAdapter', 'color': '#4153ea'}],
    'documentation': {'description': '',
     'states': [{'name': 'PREDICTED',
       'de

In [45]:
await client.ml_tool.test_plug(ref, [23.4])

2024-06-11 10:06:20 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/rules/v1/sensors/myFirstDemo/versions/0.0.1 "HTTP/1.1 200 OK"


[46.8]

In [46]:
resp = await client.registry.plugs.get_latest(PLUG_NAME)
print(f'''
version: {resp.entity.plug.version}
status: {resp.entity.status}
runtime: {resp.entity.runtime}
''')

2024-06-11 10:06:21 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/plugs/myFirstDemo "HTTP/1.1 200 OK"



version: 0.0.1
status: running
runtime: deprecated=False upgradable=False name='plug-python3' version='0.2.0'



In [47]:
# ALT use waylay.rules.plugs.test()
resp = await client.rules.plugs_execution.execute_sensor(PLUG_NAME,
    json={'properties': {'instances': [2,3,4] }},
    response_type=dict
)
resp

2024-06-11 10:06:30 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/rules/v1/sensors/myFirstDemo "HTTP/1.1 200 OK"


{'result': True,
 'state': 'PREDICTED',
 'rawData': {'predictions': [4, 6, 8]},
 'log': []}

In [48]:
await client.registry.plugs.list_versions(PLUG_NAME, select_path='entities', response_type=any)

2024-06-11 10:06:49 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/plugs/myFirstDemo/versions "HTTP/1.1 200 OK"
/opt/homebrew/Caskroom/miniconda/base/envs/ml_adapter_numpy/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:394: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


[{'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2024-06-11T08:05:51.104Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2024-06-11T08:05:51.126Z',
  'updates': [{'operation': 'create',
    'at': '2024-06-11T08:05:51.126Z',
    'by': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
    'comment': '',
    'jobs': ['740799ef-d515-4704-8718-903851c9899e$HoLoBIHR73NQKzlGJ0w64',
     '740799ef-d515-4704-8718-903851c9899e$KdOaX99-X75fYEUMhrzRs',
     '740799ef-d515-4704-8718-903851c9899e$3tYiRss-LVwzV4wfIbTpA']}],
  'status': 'running',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'plug-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': False,
  'plug': {'name': 'myFirstDemo',
   'version': '0.0.1',
   'runtime': 'plug-python3',
   'metadata': {'tags': [{'name': 'MLAdapter', 'color': '#4153ea'}],
    'documentation': {'description': '',
     'states': [{'name': 'PREDICTED',
       'description'

In [49]:
await client.ml_tool.remove(ref)

2024-06-11 10:06:54 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/plugs/myFirstDemo/versions/0.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': 'Removing plug version myFirstDemo@0.0.1',
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$8SJ1_hu3RqW81wr345eOY&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$8SJ1_hu3RqW81wr345eOY'}},
 'versions': ['0.0.1']}